In this tutorial we will be using a Gaussian Approximation Potentials to analyse results of TB DFT calculations of Si surface. Along the way we will learn about different descriptors (2b, 3b, soap) to describe local atomic environment in order to predict energies and forces of Si surface. 

In [ ]:
import os

import numpy as np
import matplotlib.pylab as plt

from quippy.potential import Potential
from quippy.descriptors import Descriptor

from ase import Atoms, units 
from ase.build import add_vacuum
from ase.lattice.cubic import Diamond
from ase.io import write

from ase.constraints import FixAtoms

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

from ase.optimize.precon import PreconLBFGS, Exp

from gap_si_surface import ViewStructure

## Part I: MD, descriptors
### Parameters

We can define some parameters here for the MD calculations:

In [ ]:
T = 1000.0 # Temperature [Kelvin]
timestep = 1.0 * units.fs

## System

Defining configuration for Si slab

In [ ]:
def build_slab(size=(1,2,2), vacuum=10.):
    # Build Si lattice.
    # lattice = Diamond('Si', directions=([1, 0, 0], [0, 1, 0], [0, 0, 1]), latticeconstant=5.44, size=size)
    lattice = Diamond('Si', latticeconstant=5.44, size=size)
    atoms = Atoms(lattice)
    

    # Fixing the bottom layer     
    bottom = atoms.positions[:,2].min()
    fixed_mask = (abs(atoms.positions[:,2] - bottom) < 2.0)
    atoms.set_constraint(FixAtoms(mask=fixed_mask))
    
    # build surface by adding space to z direction
    add_vacuum(atoms, vacuum)
    # atoms.center(vacuum=10.0, axis=2)
    
    return atoms


atoms = build_slab()
print('Number of atoms:', len(atoms))

Let's visualise the initial configuration:

In [ ]:
view = ViewStructure(atoms, repetition=(2,2,1))
view

## MD dynamics with TB DFTB

We use TB DFTB potential to calculate the potential energies and forces for the each timestep:

In [ ]:
# attach tight binding calculator
qm_pot = Potential('TB DFTB', param_filename='/opt/quip/doc/Tutorials/params.xml')
atoms.set_calculator(qm_pot)

# Thermalize atoms
MaxwellBoltzmannDistribution(atoms, 2.0 * T * units.kB)
# dynamics = VelocityVerlet(atoms, timestep)
dynamics = Langevin(atoms, timestep, T * units.kB, 0.002)
    
#attach observer to dynamics to track quantity of interest (temperature)
def print_status():    
    print('Step = {}, time = {} [fs], T = {} [K]'.format(
        dynamics.nsteps,
        dynamics.nsteps * dynamics.dt / units.fs,
        atoms.get_kinetic_energy() / (1.5 * units.kB * len(atoms))
    ))

def print_energy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
dynamics.attach(print_status, interval=10)
dynamics.attach(print_energy, interval=1)

Let's do the dynamics:

In [ ]:
dynamics.run(steps=100)

# for _ in range(150):
#     dynamics.run(steps=1)

In [ ]:
view = ViewStructure(atoms)
view

## Using descriptors

In this section we will intorduce some descriptors which could be useful for describing local atomic enviroments

You can find more details on the following webpage:
- https://libatoms.github.io/QUIP/Tutorials/quippy-descriptor-tutorial.html

## Pairwise

Here we use a simple pair distance between Silicon atoms, with a cutoff of 4.0 Angstrom. There are several descriptors that can do this, one is distance_2b, which takes a cutoff argument. Alternatively, the distance_Nb descriptor could also do this, with order=2. This is more general, order=3 is a triangle-like three-body descriptor of the three sides of a triangle of 3 atoms.

In [ ]:
desc = Descriptor("distance_Nb order=2 cutoff=4.0 n_Z=1 Zs={14}")

This descriptor is very simple: it is scalar (dimension 1), and hence only has a two (but equivalent) permutations.

In [ ]:
desc._n_perm # number of permutations

In [ ]:
desc.cutoff()

In [ ]:
desc.permutations() # array of permutation arrays


Many descriptors rely on the neighbour connectivity, so we need to call calc_connect, after setting the Atoms cutoff:

In [ ]:
desc.cutoff()

We can now calculate how many instances of this descriptor are found in an Atoms (or ASEAtoms) object:

In [ ]:
desc.count(atoms)

This also works transparently for iterables (such as lists), returning a list of the counts.

We can also calculate the actual descriptor values – in this case, the list of pairwise distances:

In [ ]:
d = desc.calc(atoms)
# for k,v in d.items():
#     print(k)
#     print(type(v))
d

Notice that the first array is called cutoff, and it supplies the value of a cutoff function, implicit in all descriptors, which takes the descriptor value to zero as the atoms approach the cutoff, i.e. in this case as the distance between the two atoms approaches the cutoff. It is more complicated for three-body and higher-body descriptors, but the final result is always a descriptor which changes smoothly with atomic positions.

Here is a histogram of the resulting descriptor array, i.e. of the interatomic distances

In [ ]:
plt.hist(d['data'], bins=40)
plt.show()

Calculate size of descriptor data:


In [ ]:
desc.sizes(atoms)

In [ ]:
n_desc, n_cross = desc.sizes(atoms)
print("n_desc=%d n_cross=%d" % (n_desc, n_cross))

In [ ]:
plt.matshow(d['data'])

## SOAP descriptor

We will describe the environment using the SOAP descriptor. The SOAP descriptor of an atomic environment is based on a spherical harmonic expansion of the neighbour density, and truncating this expansion at some maximum numer of radial (n_max) and angular (l_max) indices gives rise to some parameters. We also need to give the cutoff within which we consider the neighbour environment.

Writing the descriptor vector as $p_{ss'nn'l}$, where $s$ and $s'$ are indices that run over the different atomic species in the atom's environment, $n$ and $n'$ are radial and $l$ is an angular index.

In [ ]:
desc = Descriptor("soap cutoff=4 l_max=3 n_max=4 normalize=T atom_sigma=0.5 n_Z=1 Z={14} ")

There are now only 32 descriptors, because SOAP produces one for each atom in the structure

In [ ]:
desc.sizes(atoms)


But each descriptor now is a long vector, because it encodes the entire environment of the atom up to the cutoff. The length of the vector depends on l_max and n_max and also on the number of atom types.

In [ ]:
d = desc.calc(atoms)
d

We can visualise the values of the descriptor to have an idea of how vector looks like:

In [ ]:
plt.matshow(d['data'])

## Part II: GAP: fitting, comparing different descriptors 


## Database

Let's run the MD to generate configurations for the training:

In [ ]:
db = []
def collect_data():
    db.append(atoms.copy())

dynamics.attach(collect_data, interval=10)
dynamics.run(steps=500)

In [ ]:
# The size of the database:
len(db)

Save the results into a file for using it as an input for the fitting:

In [ ]:
# Quick fix for the shape of the 'velo' array
for at in db:
    at.arrays['velo'] = at.arrays['velo'].T

write('/tmp/atoms_db.xyz', db)

## Fitting the 2-body potential

We need to calculate the atomic energy and use it as an offset during the fitting:

In [ ]:
isolated_atom = Atoms("Si", positions=[[0,0,0]])
isolated_atom.set_calculator(qm_pot)
E0 = isolated_atom.get_potential_energy()
E0

We are going to use teach_sparse tool to build the GAP potential. For a detailed description of the available options you can run the following command.

In [ ]:
!gap_fit --help

Now we have to do actual fitting by running the following command:

Relevant parameters:
- covariance_type: Type of covariance function to use. Available: ARD_SE, DOT_PRODUCT, BOND_REAL_SP-
ACE, PP (piecewise polynomial)
- theta_uniform: Set the width of Gaussians for the ARD_SE and PP kernel, same in each dimension
- n_sparse: Number of sparse points to use in the sparsification of the Gaussian process
- delta: Set the standard deviation of the Gaussian process. Typically this would be set to the standard deviation (i.e. root mean square) of the function that is approximated with the Gaussian process.
- default_sigma: error in [energies, forces, virials, hessians]


In [ ]:
# !gap_fit at_file=dummy default_sigma={0 0 0 0} gap={distance_Nb delta=0.0 covariance_type=ARD_SE --help}

In [ ]:
!gap_fit at_file=/tmp/atoms_db.xyz \
gap={distance_Nb order=2 \
                 cutoff=5.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=15 \
                 delta=1.0} \
e0=-29.716948405885105 \
default_sigma={0.01 0.5 0.0 0.0} \
do_copy_at_file=F sparse_separate_file=F \
gp_file=/tmp/gap_2b.xml

We can load and use the fitted potential:

In [ ]:
gap2b = Potential("IP GAP", param_filename='/tmp/gap_2b.xml')

We can calculate the potential energies using the new potential and compare them with the TB DFTB ones

In [ ]:
qm_energies = [at.info['energy'] for at in db]
gap2b_energies = []
for dba in db:
    a = dba.copy()
    a.set_calculator(gap2b)
    gap2b_energies.append(a.get_potential_energy())
    

In [ ]:
Natoms = len(db[0])
plt.scatter(np.array(qm_energies)/Natoms, np.array(gap2b_energies)/Natoms)
plt.plot([E0-5.0, E0-5.0+0.05], [E0-5.0, E0-5.0+0.05], "k-")
plt.show()

We can also calculate the RMSE (in meV) as follows:

In [ ]:
np.sqrt(sum((np.array(qm_energies)/Natoms - np.array(gap2b_energies)/Natoms)**2)/len(gap2b_energies))

We can generate dimers for visualising the pairwise potential: 

In [ ]:
dimers = [Atoms("2Si", positions=[[0,0,0], [x, 0,0]]) for x in np.linspace(1.6,6,100)] 

In [ ]:
dimer_curve = []
for dim in dimers:
    dim.set_calculator(gap2b)
    dimer_curve.append(dim.get_potential_energy())

In [ ]:
plt.plot([dim.positions[1,0] for dim in dimers], np.array(dimer_curve)/2.0)
plt.show()

## Fitting the 2 and 3-body potential

In [ ]:
!gap_fit at_file=/tmp/atoms_db.xyz \
gap={distance_Nb order=2 \
                 cutoff=5.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=15 \
                 delta=1.0:\
     distance_Nb order=3 \
                 cutoff=4.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=50 \
                 delta=0.004} \
e0=-29.716948405885105 \
default_sigma={0.005 0.5 0.0 0.0} \
do_copy_at_file=F sparse_separate_file=F \
gp_file=/tmp/gap_3b.xml

We can load and use the fitted potential:

In [ ]:
gap3b = Potential("IP GAP", param_filename='/tmp/gap_3b.xml')
gap3b_energies = []
for at in db:
    a = at.copy()
    a.set_calculator(gap3b)
    gap3b_energies.append(a.get_potential_energy())
    

We can also calculate the RMSE (in meV) as follows:

In [ ]:
np.sqrt(sum((np.array(qm_energies)/Natoms - np.array(gap3b_energies)/Natoms)**2)/len(gap3b_energies))

In [ ]:
plt.scatter(np.array(qm_energies)/Natoms, np.array(gap3b_energies)/Natoms)
plt.show()

We can generate dimers for visualising the pairwise potential: 

In [ ]:
dimer_curve = []
for dim in dimers:
    dim.set_calculator(gap3b)
    dimer_curve.append(dim.get_potential_energy())
    
plt.plot([dim.positions[1,0] for dim in dimers], np.array(dimer_curve)/2.0)
plt.show()

## Fitting the 2 and 3-body  and SOAP potential

We will describe that environment using the SOAP descriptor and compare them using the SOAP kernel. Note right away that the quantum mechanically computed energies are not fully determined by the near-environment of atoms, so this is an early indication that we will be making use of the "noise" interpretation of the $\lambda$ regularization parameter: we don't expect (and don't want) our fitted function to precisely go through each datapoint.

The SOAP descriptor of an atomic environment is based on a spherical harmonic expansion of the neighbour density, and truncating this expansion at some maximum numer of radial (n_max) and angular (l_max) indices gives rise to some parameters. We also need to give the cutoff within which we consider the neighbour environment.

Writing the descriptor vector as $p_{ss'nn'l}$, where $s$ and $s'$ are indices that run over the different atomic species in the atom's environment, $n$ and $n'$ are radial and $l$ is an angular index, the kernel between two atomic environments is

$$
K(p,p') = \delta^2 \left| \sum_{ss'nn'l} p_{ss'nn'l} p'_{ss'nn'l}\right|^\zeta \equiv \delta^2 \left| {\bf p} \cdot {\bf p'}\right|^\zeta
$$

The factor of $\delta^2$ allows the setting of the scale of fitted function, relative to the error specification $\lambda$. 

In [ ]:
!gap_fit at_file=/tmp/atoms_db.xyz \
gap={distance_Nb order=2 \
                 cutoff=5.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=15 \
                 delta=1.0:\
     distance_Nb order=3 \
                 cutoff=4.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=50 \
                 delta=0.004:\
     soap cutoff=4.0 \
          covariance_type=dot_product \
          zeta=2 \
          delta=0.016 \
          atom_sigma=0.7 \
          l_max=6 \
          n_max=6 \
          n_sparse=200 \
          sparse_method=cur_points} \
e0=-29.716948405885105 \
default_sigma={0.001 0.5 0.0 0.0} \
do_copy_at_file=F sparse_separate_file=F \
gp_file=/tmp/gap_2b3bsoap.xml 2>&1 | grep -v FoX

We can load and use the fitted potential:

In [ ]:
gap_soap = Potential("IP GAP", param_filename='/tmp/gap_2b3bsoap.xml')
gap_energies = []
for at in db:
    a = at.copy()
    a.set_calculator(gap_soap)
    gap_energies.append(a.get_potential_energy())

We can also calculate the RMSE (in meV) as follows:

In [ ]:
np.sqrt(sum((np.array(qm_energies)/Natoms - np.array(gap_energies)/Natoms)**2)/len(gap_energies))

In [ ]:
plt.scatter(np.array(qm_energies)/Natoms, np.array(gap_energies)/Natoms)
plt.show()

## Using GAP for MD
We can use now use the fitted potential to do MD or geometry relaxation:

In [ ]:
#attach tight binding calculator
atoms.set_calculator(gap_soap)

# Thermalize atoms
# MaxwellBoltzmannDistribution(atoms, 2.0* T * units.kB)

# dynamics = VelocityVerlet(atoms, timestep)
dynamics = Langevin(atoms, timestep, T * units.kB, 0.002)
    
dynamics.attach(print_status, interval=10)
dynamics.attach(print_energy, interval=10)

Dynamics:

In [ ]:
dynamics.run(steps=100)

Geometry optimisation:

In [ ]:
optatoms = db[0].copy()
optatoms.set_calculator(gap_soap)
opt = PreconLBFGS(optatoms, precon=Exp(3.0))

In [ ]:
opt.run(fmax=0.01)

Let's visualise the relaxed structure:

In [ ]:
view = ViewStructure(optatoms, (2,2,1))
view.camera= 'orthographic'
view

# I hope you enjoyed it! Have a nice day!